# Keras Simple autopilot Demo

In [ ]:
# needed for matplotlib in jupyter notebook
%matplotlib inline

# import required modules
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import pandas as pd
import os
from ipyfilechooser import FileChooser


In [ ]:
fc = FileChooser()
display(fc)


In [ ]:
# prepare data
data = pd.read_csv(fc.selected)
label_movement = data.columns[0]
print(label_movement)


In [ ]:
label_movement = data.columns[0]
data = data[data[label_movement] != 'stop']
data_random_forward = data[data[label_movement] == 'forward'].sample(n=int(data.shape[0]/2))
data.drop(data_random_forward.index,inplace=True)
data[label_movement].value_counts().plot(kind='bar')
plt.title('actions')
plt.xlabel('action')
plt.ylabel('Count')

In [ ]:
# split into training and test data (10% test data)
data_test = data.sample(n=int(data.shape[0]/10))
data.drop(data_test.index,inplace=True)

#labels

conv_labels = [0 if values == 'forward' else values for values in data[label_movement] ]
conv_labels = [1 if values == 'left' else values for values in conv_labels ]
conv_labels = [2 if values == 'right' else values for values in conv_labels ]

labels = conv_labels

#labels test 
conv_labels = [0 if values == 'forward' else values for values in data_test[label_movement] ]
conv_labels = [1 if values == 'left' else values for values in conv_labels ]
conv_labels = [2 if values == 'right' else values for values in conv_labels ]

labels_test = conv_labels
print(len(labels_test))
print(len(labels))

In [ ]:
# convert image data
train = data.drop([ label_movement],axis=1).values
test = data_test.drop(label_movement,axis=1).values

train = train.reshape(train.shape[0],20,20,3)
test = test.reshape(test.shape[0],20,20,3)

## Data preparation

In [ ]:
# create a smaller test set

x_train = train.astype('float32') / 255
x_test = test.astype('float32') / 255

# convert classes to one-hot encoding
num_classes = 3
y_train = keras.utils.to_categorical(labels, num_classes)
y_test = keras.utils.to_categorical(labels_test, num_classes)
print("one-hot labels:\n", y_train[:300])


## Training

In [ ]:
# create model
model = Sequential([
    Input(shape=(20, 20, 3)),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# compile model
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
# fit model to training data
model.fit(x_train, y_train, validation_split=0.2, epochs=15)

## Evaluation

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test)

print('Test accuracy:', score)

In [ ]:
model.save(os.path.join('..', 'models', 'my-keras-model.keras'))